In [1]:
import cv2
import mediapipe as mp
import math
import joblib
import numpy as np

In [2]:
model = joblib.load("yoga_pose_model.pkl")
scaler = joblib.load("scaler.pkl")


In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [4]:
def calculate_angle(a, b, c):
    ba = (a[0]-b[0], a[1]-b[1])
    bc = (c[0]-b[0], c[1]-b[1])
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cosine_angle = dot_product / (mag_ba * mag_bc + 1e-6)
    return math.degrees(math.acos(max(min(cosine_angle, 1), -1)))

def calculate_distance(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

def extract_features_from_landmarks(landmarks):
    # Angles
    left_shoulder_angle = calculate_angle(landmarks[13], landmarks[11], landmarks[23])
    right_shoulder_angle = calculate_angle(landmarks[14], landmarks[12], landmarks[24])
    left_elbow_angle = calculate_angle(landmarks[11], landmarks[13], landmarks[15])
    right_elbow_angle = calculate_angle(landmarks[12], landmarks[14], landmarks[16])
    left_hip_angle = calculate_angle(landmarks[11], landmarks[23], landmarks[25])
    right_hip_angle = calculate_angle(landmarks[12], landmarks[24], landmarks[26])
    left_knee_angle = calculate_angle(landmarks[23], landmarks[25], landmarks[27])
    right_knee_angle = calculate_angle(landmarks[24], landmarks[26], landmarks[28])

    # Distances
    shoulder_distance = calculate_distance(landmarks[11], landmarks[12])
    hip_distance = calculate_distance(landmarks[23], landmarks[24])
    l_shoulder_to_hip = calculate_distance(landmarks[11], landmarks[23])
    r_shoulder_to_hip = calculate_distance(landmarks[12], landmarks[24])
    l_shoulder_to_ankle = calculate_distance(landmarks[11], landmarks[27])
    r_shoulder_to_ankle = calculate_distance(landmarks[12], landmarks[28])

    return [
        left_shoulder_angle, right_shoulder_angle,
        left_elbow_angle, right_elbow_angle,
        left_hip_angle, right_hip_angle,
        left_knee_angle, right_knee_angle,
        shoulder_distance, hip_distance,
        l_shoulder_to_hip, r_shoulder_to_hip,
        l_shoulder_to_ankle, r_shoulder_to_ankle
    ]

In [5]:
header = [
    "left_shoulder_angle", "right_shoulder_angle",
    "left_elbow_angle", "right_elbow_angle",
    "left_hip_angle", "right_hip_angle",
    "left_knee_angle", "right_knee_angle",
    "shoulder_distance", "hip_distance",
    "l_shoulder_to_hip", "r_shoulder_to_hip",
    "l_shoulder_to_ankle", "r_shoulder_to_ankle"
]

In [ ]:
ideal_pose_angles = {
    "downdog": {
        "left_shoulder_angle": (160, 180),
        "right_shoulder_angle": (160, 180),
        "left_hip_angle": (170, 180),
        "right_hip_angle": (170, 180)
    },
    "goddess": {
        "left_knee_angle": (80, 100),
        "right_knee_angle": (80, 100),
        "left_hip_angle": (90, 110),
        "right_hip_angle": (90, 110)
    },
    "plank": {
        "left_elbow_angle": (170, 180),
        "right_elbow_angle": (170, 180),
        "left_hip_angle": (170, 180),
        "right_hip_angle": (170, 180)
    },
    "side plank": {
        "left_elbow_angle": (170, 180),
        "right_elbow_angle": (170, 180),
        "left_hip_angle": (170, 180),
        "right_hip_angle": (170, 180)
    },
    "tree": {
    "left_knee_angle": (40, 70),   # bent leg
    "right_knee_angle": (160, 180), # straight leg
    "left_shoulder_angle": (160, 180),
    "right_shoulder_angle": (160, 180),
    "left_elbow_angle": (160, 180),
    "right_elbow_angle": (160, 180)
},
    "warrior": {
        "left_knee_angle": (85, 95),
        "right_knee_angle": (170, 180),
        "left_hip_angle": (160, 175),
        "right_hip_angle": (160, 175)
    }
}

In [7]:
def get_feedback(pose_name, features_dict):
    pose_name = pose_name.lower()
    if pose_name not in ideal_pose_angles:
        return "No feedback available"
    
    feedback_list = []
    
    for joint, (low, high) in ideal_pose_angles[pose_name].items():
        angle = features_dict.get(joint, None)
        
        if angle is None:
            feedback_list.append(f"Could not detect {joint.replace('_angle','').replace('_',' ')}")
            continue
        
        joint_name = joint.replace("_angle", "").replace("_", " ")
        
        if angle < low:
            feedback_list.append(f"Increase your {joint_name}")
        elif angle > high:
            feedback_list.append(f"Decrease your {joint_name}")
    
    return "; ".join(feedback_list) if feedback_list else "Perfect pose!"



# Predict pose probabilities
probs = model.predict_proba(scaled_features)[0]

# Map class names to probabilities
pose_probs = dict(zip(model.classes_, probs))

# Print in console for debugging
print(pose_probs)

# Choose the pose with the highest probability
predicted_pose = max(pose_probs, key=pose_probs.get)


In [13]:
cap = cv2.VideoCapture(0) 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

   
    cv2.rectangle(frame, (5, 5), (400, 130), (0, 0, 0), -1)

    if results.pose_landmarks:
        landmarks = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]

        features = extract_features_from_landmarks(landmarks)
        scaled_features = scaler.transform([features])  # scale to match training

        probs = model.predict_proba(scaled_features)[0]
        max_prob = max(probs)
        predicted_pose = model.classes_[np.argmax(probs)]
        
        # Sitting means hips and knees almost aligned
        if abs(features_dict["left_hip_angle"] - features_dict["left_knee_angle"]) < 15 and \
            abs(features_dict["right_hip_angle"] - features_dict["right_knee_angle"]) < 15:
            predicted_pose = "Unknown"


        if max_prob < 0.8:
            predicted_pose = "Unknown"
            feedback = "Cannot determine pose confidently."
            overall_status = "Unknown"
            status_color = (0, 255, 255)
        else:
            features_dict = dict(zip(header, features))
            feedback = get_feedback(predicted_pose, features_dict)

            if feedback == "Perfect pose!":
                overall_status = "Good"
                status_color = (0, 255, 0)
            else:
                overall_status = "Bad"
                status_color = (0, 0, 255)

        mp_drawing.draw_landmarks(
            frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
        )

        cv2.putText(frame, f'Pose: {predicted_pose}', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f'Status: {overall_status}', (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
        cv2.putText(frame, feedback, (10, 110), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    else:
        cv2.putText(frame, "Pose: Unknown", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Yoga Pose Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\valida

In [11]:
cap = cv2.VideoCapture(0)  # 0 = webcam. Use path if it's a video file.

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        # Extract landmark coordinates
        landmarks = []
        # Extract landmark coordinates
        landmarks = []
        for lm in results.pose_landmarks.landmark:
            landmarks.append((lm.x, lm.y))  # keep normalized coords instead of pixels

        # Get feature vector
        features = extract_features_from_landmarks(landmarks)
        scaled_features = scaler.transform([features])  # scale to match training

        # Predict pose
        predicted_pose = model.predict(scaled_features)[0]

        # Create dict for feedback
        # Create dict for feedback
        features_dict = dict(zip(header, features))
        feedback = get_feedback(predicted_pose, features_dict)

        # Determine overall feedback label
        if feedback == "Perfect pose!":
            overall_status = "Good"
            status_color = (0, 255, 0)  # Green
        else:
            overall_status = "Bad"
            status_color = (0, 0, 255)  # Red

        # Draw pose landmarks on frame
        mp_drawing.draw_landmarks(
            frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
        )

        # Display prediction and feedback
        cv2.putText(frame, f'Pose: {predicted_pose}', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f'Status: {overall_status}', (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
        cv2.putText(frame, feedback, (10, 110), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    else:
        # If no pose detected
        cv2.putText(frame, "Pose: Unknown", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Yoga Pose Detection', frame)

    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\valida

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Function to calculate joint angle
def calculate_angle(a, b, c):
    a = np.array(a)  # first point
    b = np.array(b)  # mid point
    c = np.array(c)  # end point
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [7]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark
        landmarks = [(lm[i].x, lm[i].y) for i in range(33)]

        # Extract features and predict
        features = np.array([extract_features_from_landmarks(landmarks)])
        predicted_pose = model.predict(features)[0]

        # Pose correction
        corrections = []
        if predicted_pose in ideal_pose_angles:
            for angle_name, (low, high) in ideal_pose_angles[predicted_pose].items():
                feature_index = header.index(angle_name)
                current_value = features[0][feature_index]
                if not (low <= current_value <= high):
                    corrections.append(f"Adjust {angle_name}: {int(current_value)}° (want {low}-{high}°)")

        # Draw pose + feedback
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(frame, f"Pose: {predicted_pose}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        y_offset = 80
        for correction in corrections:
            cv2.putText(frame, correction, (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            y_offset += 30

    cv2.imshow("Yoga Pose Detection + Correction", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\SUSHMITA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have v